<font size="6">LDAによるニュース分類</font>

Source: https://dev.classmethod.jp/machine-learning/2017ad_20171221_lda_python/

機械学習_潜在意味解析_pythonで実装

<font size="5">データのロード</font>

In [1]:
'''1.必要なモジュールとデータセットの準備
sklearnに用意されている「ニュース」のデータセットを利用します
'''
 
# モジュールのインポート
from sklearn.datasets import fetch_20newsgroups 
 
'''
sklearnに用意されている「ニュース」のデータセットを利用します。
'''
 
# カテゴリーの絞り込み
categories = ['rec.sport.baseball', 'rec.sport.hockey', 'comp.sys.mac.hardware', 'comp.windows.x']
 
# トレーニング用、検証用のデータセット作成
twenty_train = fetch_20newsgroups(subset='train',categories=categories, shuffle=True, random_state=42)
twenty_test = fetch_20newsgroups(subset='test',categories=categories, shuffle=True, random_state=42)
 
# データの中身確認
print(twenty_train.target_names) # カテゴリの確認(どんなカテゴリがあるのか)
print (len(twenty_train.data)) # ニュースデータが何本あるのか（2,368本のニュースデータ）
print (len(twenty_test.data)) # ニュースデータが何本あるのか（1,576本のニュースデータ）
print("\n".join(twenty_train.data[0].split("\n"))) #データの内容を確認(1個の文書の中身を確認)
print(twenty_train.target_names[twenty_train.target[0]]) #データのカテゴリーを確認

['comp.sys.mac.hardware', 'comp.windows.x', 'rec.sport.baseball', 'rec.sport.hockey']
2368
1576
From: pkeenan@s.psych.uiuc.edu (Patricia Keenan)
Subject: Re: Quick easy question!
Organization: UIUC Department of Psychology
Lines: 16

rauser@fraser.sfu.ca (Richard John Rauser) writes:



>   Here's an easy question for someone who knows nothing about baseball...

>   What city do the California Angels play out of?

   Anaheim.

>-- 
>Richard J. Rauser        "You have no idea what you're doing."
>rauser@sfu.ca            "Oh, don't worry about that. We're professional
>WNI                          outlaws - we do this for a living."
>-----------------
>"Remember, no matter where you go, there you are." -Dr.Banzai

rec.sport.baseball


<font size="5">データの前処理</font>

In [2]:
'''2.前処理
前処理で実施したものは下記の４点。
・文字を全て小文字化
・stop words削除
・全文書の１割以上の文書に出現する単語の削除
・5回以上出現する単語のみ
'''
 
# tf-idf計算した時の単語数を確認してみる
from sklearn.feature_extraction.text import TfidfVectorizer
 
tfidf_vec = TfidfVectorizer(lowercase=True, stop_words='english', max_df = 0.1, min_df = 5).fit(twenty_train.data) # 
X_train = tfidf_vec.transform(twenty_train.data) 
X_test = tfidf_vec.transform(twenty_test.data) 
 
print('tf-idfの計算結果の概要\n:{}'.format(repr(X_train))) # 2,368行7,562列の疎行列
 
# 各単語の中身を確認する
feature_names = tfidf_vec.get_feature_names()
print('\n最初の３０個の単語\n:{}'.format(feature_names[:30]))
print('\n500-530\n:{}'.format(feature_names[500:530]))
print('\n1000-1030\n:{}'.format(feature_names[1000:1030]))

tf-idfの計算結果の概要
:<2368x7562 sparse matrix of type '<class 'numpy.float64'>'
	with 159495 stored elements in Compressed Sparse Row format>

最初の３０個の単語
:['00', '000', '0062', '00mbstultz', '01', '02', '0200', '0223', '024222', '03', '030', '04', '040', '0400', '05', '051', '06', '065', '0666', '07', '08', '09', '0ha', '100', '1000', '101', '1010', '102', '1020', '1024']

500-530
:['636', '637', '64', '640', '643', '645', '647', '649', '64k', '65', '650', '653', '655', '656', '657', '66', '660', '664', '666', '667', '66mhz', '67', '672', '675', '678', '68', '680', '68000', '68020', '68030']

1000-1030
:['ask', 'asked', 'asking', 'asks', 'aspect', 'aspects', 'ass', 'assat', 'assembled', 'assessment', 'asshole', 'assholes', 'assist', 'assistance', 'assistant', 'assists', 'associated', 'associates', 'association', 'assume', 'assumed', 'assumes', 'assuming', 'ast', 'astro', 'astronomy', 'astros', 'asynchronous', 'atd', 'athena']


<font size="5">モデルの生成と評価(注意！実行にかなり時間がかかる)</font>

In [3]:
''' 3.モデル生成と評価
「perplexity」を評価指標とします。
パラメータは、「トピック数」と「反復回数」を指定します。
本来は「反復回数」は推移を確認したほうがいいとは思うのですが...
'''
 
# 3-1.パラメータの調整
from sklearn.decomposition import LatentDirichletAllocation
import time
n_comp = [2, 3, 4, 5, 6,10] # トピック数の指定
max_iter = [50, 100, 300, 500] # 反復回数の指定
 
for comp in n_comp:
    for max_i in max_iter:
 
        # 実行時間計測開始
        start = time.time()
 
        lda =LatentDirichletAllocation(n_components=comp,  max_iter=max_i, learning_method='batch', random_state=0, n_jobs=-1)
        lda_fit = lda.fit(X_train)
        lda_perp = lda.perplexity(X_test)
 
        # 実行時間計測終了
        elapsed_time = time.time() - start 
 
        # 結果の表示
        print("トピック数:{0},反復回数:{1}回,処理時間:{2}分,exp(-1. * log-likelihood per word)(perplexity):{3}".format(comp, max_i, round(elapsed_time/60, 2), round(lda_perp, 2)))


トピック数:2,反復回数:50回,処理時間:0.38分,exp(-1. * log-likelihood per word)(perplexity):11768.51
トピック数:2,反復回数:100回,処理時間:0.62分,exp(-1. * log-likelihood per word)(perplexity):11768.57
トピック数:2,反復回数:300回,処理時間:1.66分,exp(-1. * log-likelihood per word)(perplexity):11768.57
トピック数:2,反復回数:500回,処理時間:2.61分,exp(-1. * log-likelihood per word)(perplexity):11768.57
トピック数:3,反復回数:50回,処理時間:0.37分,exp(-1. * log-likelihood per word)(perplexity):17218.32
トピック数:3,反復回数:100回,処理時間:0.62分,exp(-1. * log-likelihood per word)(perplexity):17215.31
トピック数:3,反復回数:300回,処理時間:1.66分,exp(-1. * log-likelihood per word)(perplexity):17210.34
トピック数:3,反復回数:500回,処理時間:2.64分,exp(-1. * log-likelihood per word)(perplexity):17210.34
トピック数:4,反復回数:50回,処理時間:0.36分,exp(-1. * log-likelihood per word)(perplexity):19769.88
トピック数:4,反復回数:100回,処理時間:0.62分,exp(-1. * log-likelihood per word)(perplexity):19660.65
トピック数:4,反復回数:300回,処理時間:1.51分,exp(-1. * log-likelihood per word)(perplexity):19552.25
トピック数:4,反復回数:500回,処理時間:2.57分,exp(-1. * log-likelihood per word)(perp

<font size="5">トピックの確認と可視化</font>

In [5]:
''' 3.モデル生成と評価
上記の結果から、トピック数を2とします。
また、反復回数は100回くらいで良さそうですね
'''
 
# 3-2.先ほど求めたパラメータに応じたモデルの生成
 
# データの用意（トレーニング用、検証用と分けずに全セット）
twenty_train = fetch_20newsgroups(categories=categories, shuffle=True, random_state=42)
X = tfidf_vec.transform(twenty_train.data) 
 
# クラスの生成
from sklearn.decomposition import LatentDirichletAllocation
lda =LatentDirichletAllocation(n_components=2,  max_iter=100, learning_method='batch', random_state=0, n_jobs=-1)
 
# モデルの生成
lda.fit(X)
lda_X = lda.transform(X)

In [7]:
''' 4.トピックの確認
各トピックに実際にどのような単語、文書が割り当てられているのかを確認します。
'''
import numpy as np

# 4-2.各トピックごとにどんな文書が割り当てられているか
 
# トピック０
topic_0 = np.argsort(lda_X[:, 0])[::-1] # このトピックでもっとも重要度が高い順にソート
 
for i in range(0,2):
    print(twenty_train.data[topic_0[i]].split('.')[:30])
    print('\n' + '------------------------------')

['From: tedward@cs', 'cornell', 'edu (Edward [Ted] Fischer)\nSubject: Old Predictions to laugh at', '', '', '\nSummary: LONG!\nOrganization: Cornell Univ', ' CS Dept, Ithaca NY 14853\nLines: 404\n\n\nOops!  I came across this file from last year', '  Thought you might\nenjoy some of these thoughts', '  The predictions were made on the\ndate indicated', '  They are largely out of order', '\n\n------------------------------------------------------------------------------\n\nJune 11, 1992\ntedward@cs', 'cornell', 'edu (ME!)\n>What have I done?  I computed the "expected winning percentage" for\n>each team from their OBP, total bases, and runs allowed', '  I use the\n>basic RC formula and the pythagorean projection', '  I then compare this\n>with their actual winning percentage', '  All stats through June 7', '\n>\n>Team           OBP    TB    RA     W     L    XWP  Diff\n>baltimore    0', '351   768   199    33    21  0', '647   -36\n>boston       0', '334   580   176    26    25  0', '548

<font size="5">テストデータによる評価</font>

In [8]:
'''5.テストデータを入れて確認
yahooニュースをgoogle翻訳で英語にしたものを入れてみた
'''
 
# 5-1.大谷選手がエンゼルス入団を決めたニュースでテスト
 
test = [
'Nippon Ham\'s Shohei Otani pitcher (23) who was aiming for a major transfer in the posting system decided to contract with Angels. On August 8, agent Mr. Barzero announced. In addition, Angels also announced the following statement.',
'The U.S. major league, Angels announced that the shoulder number of Shohei Otani pitcher (23) whose entry was decided will be "17".', 
]
 
 
X_test = tfidf_vec.transform(test) 
lda_test = lda.transform(X_test)
print(lda_test)

[[0.83430136 0.16569864]
 [0.83230705 0.16769295]]


In [9]:
'''5.テストデータを入れて確認
yahooニュースをgoogle翻訳で英語にしたものを入れてみた
'''
 
# 5-2.macpro発売、macへの不正ログインのニュース
 
test=[
'Apple announced on December 12 (local time) that December 14 will release a new desktop iMac Pro that will be "the most powerful Mac ever". Although the selling price in Japan is unpublished, it is 4,999 dollars (about 570,000 yen) in the United States.',
'Developer Lemi Orhan Ergin discovered a vulnerability in macOS High Sierra 10.13.1. When you click on the lock button from "User and group" in the system environment setting and enter "user name" and password in order to unlock the preference setting, enter "root" for the user name, enter the password Even without it, the lock is released.'
]
 
 
X_test = tfidf_vec.transform(test) 
lda_test = lda.transform(X_test)
print(lda_test)

[[0.36815481 0.63184519]
 [0.12606953 0.87393047]]
